# LIBRARIES & FUNCTIONS

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

import pydicom

import json

from Gafchromic import Radiochromic_RB
from Gafchromic_multichannel import Radiochromic_multilinear

import time



output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS #



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray



# reading dose matrix in axial plane at isocenter 
#    from dose and plan dicom files : #
# @params:
#  doseFilename : filename of the dicom file containing the dose matrix
#  planFilename : filename of the dicom file containing the RT plan
#  verbose : writes the print output ?
def readDoseMatrix_AxialPlane_Isocenter(doseFilename, planFilename, offset=0, verbose=False):

    ds_dose = pydicom.read_file(doseFilename)
    ds_plan = pydicom.read_file(planFilename)
    
    if verbose: print('Dose files read!')

    
    # Isocenter Coordinates from plan file:
    iso_x=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[0]
    iso_y=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[1]
    iso_z=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[2]

    if verbose: print ('  > isocenter coordinates = (', iso_x, ";", iso_y, ";", iso_z, ")")


    # Dose image coordinates from dose file:
    position_image=list(ds_dose['0020','0032'].value) # +x left, +y post, +z head   
    position_image_x=list(ds_dose['0020','0032'].value)[0]
    position_image_y=list(ds_dose['0020','0032'].value)[1]
    position_image_z=list(ds_dose['0020','0032'].value)[2]

    if verbose: print ('  > Dose image position = (', position_image_x, ";", position_image_y, ";", position_image_z, ")")


    # Dose matrix in axial plane at isocenter:
    dim_dose= ds_dose.pixel_array.shape # disposition y, z, x?
    
    if verbose: print ('  > dose matrix size = ', dim_dose)

    pixel_spacing_x=(ds_dose['0028','0030'].value[0]) #definition de la résolution de la matrice de dose
    pixel_spacing_z=(ds_dose['0028','0030'].value[1])
    pixel_spacing_y=(ds_dose['3004','000C'].value[1])
    
    if verbose: print ('  > pixel spacing = (', pixel_spacing_x, ";", pixel_spacing_y, ";", pixel_spacing_z, ")")

    coord_iso_dose = (round((iso_x-position_image_x)/pixel_spacing_x),
                      round((iso_y-position_image_y)/pixel_spacing_y),
                      round((iso_z-position_image_z)/pixel_spacing_z))

    if verbose: print ('  > isocenter coodinates in dose matrix =', coord_iso_dose)

    doseimg = ds_dose.pixel_array[:,coord_iso_dose[1]+offset,:] * ds_dose.DoseGridScaling * 100 #en cGy

    if verbose: print ('  > maximum dose :', np.amax(doseimg))

    return (doseimg, dim_dose[0], dim_dose[2], pixel_spacing_x, pixel_spacing_z)




# Creating a calibration model for a line. It results parameters of the fit
#   for the RB vs GB curve, and the parameters of the fit for the dose vs RB&GB
#   curve.
#   
#  <!> funcDose et funcRBGB must be defined first !
#
# @params:
#  arrayrb : image matrix R/B
#  arraygb : image matrix G/B
#  arraydose : dose matrix 
#  column : column to be used
#  beginLine : line where to begin process
#  endLine : line where to end process
#  dispParameters : prints parameters found
#  dispGraphs : displays graphs
def createLineCal_RBGBModel(arrayrb, arraygb, arraydose, column, beginLine, endLine, w=0,
                       dispParameters=False, dispGraphs=False):
    
    # crops the matrices:
    if w==0:
        rb = array_rb[beginLine:endLine, column]
        gb = array_gb[beginLine:endLine, column]
        dose = arraydose[beginLine:endLine, column]
    else:
        rb = np.mean(array_rb[beginLine:endLine, column-w:column+w], axis=1)
        gb = np.mean(array_gb[beginLine:endLine, column-w:column+w], axis=1)
        dose = np.mean(arraydose[beginLine:endLine, column-w:column+w], axis=1)
    
    # Dose vs RB&GB fitting:
    nbLines = rb.shape[0]
    rbgb = np.zeros((nbLines,2))
    rbgb[:,0] = rb
    rbgb[:,1] = gb

    paramsDose, params_covDose = optimize.curve_fit(funcDose, rbgb, dose)

    
    # Display parameters models:
    if dispParameters:
        print('Parameters Dose vs RB & GB:')
        for i in range(len(paramsDose)):
            print('   - parameter', i, ':', paramsDose[i])

    # Display graph results
    if dispGraphs:
        
        # Image view: 
        p2 = figure(plot_width=300, plot_height=int(300*arrayrb.shape[0]/arrayrb.shape[1]), 
                   title='', toolbar_location="above")

        p2.image(image=[arrayrb], x=0, y=0, dw=arrayrb.shape[1], dh=arrayrb.shape[0], palette="Plasma256")
        p2.line((column, column), (beginLine, endLine), line_alpha=0.7, line_color="white")
        if w>0: 
            p2.rect(x=column, y=int(beginLine+(endLine-beginLine)/2), width=2*w+1, height=endLine-beginLine, 
                    fill_color='white', fill_alpha=0.3, line_alpha=0.3, line_color="white")
        

        # RB & GB vs dose:
        p3 = figure(plot_width=600, plot_height=600, title="RB & GB vs dose", toolbar_location="above")
        p3.circle(rb, dose, size=3, color='darkslategrey', alpha=0.3, legend_label="R/B") 
        p3.circle(gb, dose, size=3, color='gold', alpha=0.3, legend_label="G/B") 

        # Plots the results of calibration on this line ()
        p4 = figure(plot_width=450, plot_height=450, title="Dose calculated vs real", toolbar_location="above")
        p4.line(dose, funcDose(rbgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], paramsDose[4]), 
                line_width=2, line_color='firebrick')
        p4.line(dose, dose, line_width=1, line_color='black', line_dash='dotted')

        p5 = figure(plot_width=450, plot_height=450, title="Dose calculation Error in %", toolbar_location="above")
        p5.line(dose,(dose-funcDose(rbgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], \
                                    paramsDose[4]))/dose*100, line_width=2, line_color='firebrick')
        p5.line(dose,2 , line_width=1, line_color='black', line_dash='dotted')
        p5.line(dose,-2 , line_width=1, line_color='black', line_dash='dotted')

        # grid = gridplot([p1, p2, p3], ncols=2)
        grid1 = gridplot([[p3, p2]])
        grid2 = gridplot([[p4,p5]])

        show(grid1)
        show(grid2)

    # Returns parameters:
    return paramsDose




# Converts an image to dose using RBGB models
#   
# @params:
#  arr : image array
#  models : dictionary of RBGB model
#  meanWidth: mean of the line i in [:,i-meanWidth:i+meanWidth]
def convertToDose(arr, models, meanWidth=0):
    
    rb = arr[:,:,0]/arr[:,:,2]
    gb = arr[:,:,1]/arr[:,:,2]
    
    doseImg = np.zeros(rb.shape)
    
    rbgbmodels = models["RBGBcurve"]
    for i in range(meanWidth, rb.shape[1]-meanWidth):
        if meanWidth==0:
            [index, err] = findBestModel(rb[:,i], gb[:,i], rbgbmodels, dispGraph=False)
        else:
            [index, err] = findBestModel(np.mean(rb[:,i-meanWidth:i+meanWidth], axis=1), 
                                         np.mean(gb[:,i-meanWidth:i+meanWidth], axis=1), 
                                         rbgbmodels, dispGraph=False)

        
        rbgb = np.zeros((rb.shape[0],2))
        rbgb[:,0] = rb[:,i]
        rbgb[:,1] = gb[:,i]
        mod = models["DoseCurve"][index]
        doseImg[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])
    
    return doseImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue', visibleLegend = True, legendPos = 'bottom_left'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")

    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)

    p3.legend.click_policy="hide"
    p3.legend.location = legendPos
    
    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    p4.legend.click_policy="hide"
    p4.legend.location = legendPos

    if not visibleLegend:
        p3.legend.visible=False
        p4.legend.visible=False
        
    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
    
# Applies streak correction to RGB image
#   
# @params:
#  array: image array on which to perform streak correction.
def applyStreakCorrection(array):
        
        # First pixels sum and normalization:
        firstPixAvg_r = np.sum(array[0:10,:,0] ,axis=0)
        firstPixAvg_g = np.sum(array[0:10,:,1] ,axis=0)
        firstPixAvg_b = np.sum(array[0:10,:,2],axis=0)

        firstPixAvg_r = firstPixAvg_r / np.mean(firstPixAvg_r)
        firstPixAvg_g = firstPixAvg_g / np.mean(firstPixAvg_g)
        firstPixAvg_b = firstPixAvg_b / np.mean(firstPixAvg_b)


        # Last pixels sum and normalization:
        lastPixAvg_r = np.sum(array[-9:,:,0] ,axis=0)
        lastPixAvg_g = np.sum(array[-9:,:,1] ,axis=0)
        lastPixAvg_b = np.sum(array[-9:,:,2],axis=0)

        lastPixAvg_r = lastPixAvg_r / np.mean(lastPixAvg_r)
        lastPixAvg_g = lastPixAvg_g / np.mean(lastPixAvg_g)
        lastPixAvg_b = lastPixAvg_b / np.mean(lastPixAvg_b)


        # Streak correction image:
        #  This correction must be done on an uncroped image
        streakCorrImage = np.zeros(array.shape)
        x = [0, array.shape[1]-1]

        for i in range(array.shape[1]):
            allpix_y = range(array.shape[0])
            y0 = [firstPixAvg_r[i], lastPixAvg_r[i]]
            streakCorrImage[:,i,0] = 1 / np.interp(allpix_y, x, y0)
    
            y1 = [firstPixAvg_g[i], lastPixAvg_g[i]]
            streakCorrImage[:,i,1] = 1 / np.interp(allpix_y, x, y1)

            y2 = [firstPixAvg_b[i], lastPixAvg_b[i]]
            streakCorrImage[:,i,2] = 1 / np.interp(allpix_y, x, y2)
    
    
        # Apply streak correction to the image:
        return array * streakCorrImage

    
    
    
# returns img2 registered to img1
#   
# @params:
#  img1: fixed img
#  img2: moving img to register
def registerImgs(img1, img2, nbHistoBins=20, nbiter=100, convMinValue=1e-8):
    
    fixedimg = sitk.Image(img1.shape[1], img1.shape[0], sitk.sitkFloat32)
    for i in range(img1.shape[1]):
        for j in range(img1.shape[0]):
            fixedimg.SetPixel(i, j, float(img1[j,i]))

    movingimg = sitk.Image(img2.shape[1], img2.shape[0], sitk.sitkFloat32)
    for i in range(img2.shape[1]):
        for j in range(img2.shape[0]):
            movingimg.SetPixel(i, j, float(img2[j,i]))

#     initial_transform = sitk.TranslationTransform(fixedimg.GetDimension())
    initial_transform = sitk.CenteredTransformInitializer(fixedimg,
                                                        movingimg,
                                                        sitk.Euler2DTransform(),
                                                        sitk.CenteredTransformInitializerFilter.GEOMETRY)

    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsJointHistogramMutualInformation(numberOfHistogramBins=nbHistoBins)
#     registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingStrategy(registration_method.REGULAR)
    registration_method.SetMetricSamplingPercentage(0.1)
    registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=nbiter, 
                                                      convergenceMinimumValue=convMinValue, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    final_transform = registration_method.Execute(sitk.Cast(fixedimg, sitk.sitkFloat32), 
                                                  sitk.Cast(movingimg, sitk.sitkFloat32))
    itkfinalimg = sitk.Resample(movingimg, fixedimg, final_transform, sitk.sitkBSplineResamplerOrder5, 
                             0.0, movingimg.GetPixelID())

    finalimg = sitk.GetArrayFromImage(itkfinalimg)
    
    return finalimg


# IMG READING & REGISTRATION

In [8]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers



# Variables:
m_path = 'G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/newWedge/'
m_nbOfFiles = 1
m_firstNb = 5

m_GafFilesNameLeft = "24h/pos01_0"
m_rectL = [6,226,3,193]

m_GafFilesNameCenter = "24h/pos02_0"
m_rectC = [5,225,60,250]

m_GafFilesNameRight = "24h/pos03_0"
m_rectR = [7,227,117,307]

m_fileExtension = ".tif"

m_doseFileName = "RD_newWedge_1mm.dcm"
m_planFileName =  "RP_newWedge_1mm.dcm"



# to compensate for the problems in field size (change DSD and inverse square law)
dsd = 1000
m_doseOffset = -7
m_doseFactor = (dsd-m_doseOffset)*(dsd-m_doseOffset)/(dsd*dsd)

m_splineFile = "G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_normalCalib.txt"
m_rectDose = [80,320,100,300]

m_dimViewer = 600




In [9]:
# READS THE FILES:

# Disp images?
visu = False
dispSize = 450

# Reads the dose img:
(doseimg, dimx, dimy, pixsizex, pixsizey) = readDoseMatrix_AxialPlane_Isocenter(
    m_path+m_doseFileName, m_path+m_planFileName, offset=m_doseOffset, verbose=False)

calcDose = doseimg[m_rectDose[0]:m_rectDose[1],m_rectDose[2]:m_rectDose[3]]*m_doseFactor#np.flip(doseimg)

print('   >>> dOSE rEAD !')


# Reads and convert to dose the gafchromic film:
try:
    g = Radiochromic_RB(m_path+m_GafFilesNameLeft, m_firstNb, m_nbOfFiles, addMethod='mean')
    measDoseL = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    rgbArrL = g.getArray()

    g = Radiochromic_RB(m_path+m_GafFilesNameCenter, m_firstNb, m_nbOfFiles, addMethod='mean')
    measDoseC = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    rgbArrC = g.getArray()
    
    g = Radiochromic_RB(m_path+m_GafFilesNameRight, m_firstNb, m_nbOfFiles, addMethod='mean')
    measDoseR = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    rgbArrR = g.getArray()

except ValueError as err:
    print('Erreur: ' + err)
    
    
rArrL = rgbArrL[:,:,0]
rArrL = subSampleDataArray(rArrL, 10)
rArrL_crop = rArrL[m_rectL[0]:m_rectL[1],m_rectL[2]:m_rectL[3]]

gArrL = rgbArrL[:,:,1]
gArrL = subSampleDataArray(gArrL, 10)
gArrL_crop = gArrL[m_rectL[0]:m_rectL[1],m_rectL[2]:m_rectL[3]]

bArrL = rgbArrL[:,:,2]
bArrL = subSampleDataArray(bArrL, 10)
bArrL_crop = bArrL[m_rectL[0]:m_rectL[1],m_rectL[2]:m_rectL[3]]

measDoseL = subSampleDataArray(measDoseL, 10)
measDoseL = measDoseL[m_rectL[0]:m_rectL[1],m_rectL[2]:m_rectL[3]]


rArrC = rgbArrC[:,:,0]
rArrC = subSampleDataArray(rArrC, 10)
rArrC_crop = rArrC[m_rectC[0]:m_rectC[1],m_rectC[2]:m_rectC[3]]

gArrC = rgbArrC[:,:,1]
gArrC = subSampleDataArray(gArrC, 10)
gArrC_crop = gArrC[m_rectC[0]:m_rectC[1],m_rectC[2]:m_rectC[3]]

bArrC = rgbArrC[:,:,2]
bArrC = subSampleDataArray(bArrC, 10)
bArrC_crop = bArrC[m_rectC[0]:m_rectC[1],m_rectC[2]:m_rectC[3]]

measDoseC = subSampleDataArray(measDoseC, 10)
measDoseC = measDoseC[m_rectC[0]:m_rectC[1],m_rectC[2]:m_rectC[3]]


rArrR = rgbArrR[:,:,0]
rArrR = subSampleDataArray(rArrR, 10)
rArrR_crop = rArrR[m_rectR[0]:m_rectR[1],m_rectR[2]:m_rectR[3]]

gArrR = rgbArrR[:,:,1]
gArrR = subSampleDataArray(gArrR, 10)
gArrR_crop = gArrR[m_rectR[0]:m_rectR[1],m_rectR[2]:m_rectR[3]]

bArrR = rgbArrR[:,:,2]
bArrR = subSampleDataArray(bArrR, 10)
bArrR_crop = bArrR[m_rectR[0]:m_rectR[1],m_rectR[2]:m_rectR[3]]

measDoseR = subSampleDataArray(measDoseR, 10)
measDoseR = measDoseR[m_rectR[0]:m_rectR[1],m_rectR[2]:m_rectR[3]]

print('   >>> gAFCHROMIC fILMS rEAD !')


# shows dose images:
if visu:
    p1 = figure(plot_width=dispSize, plot_height=int(dispSize*calcDose.shape[0]/calcDose.shape[1]), 
               title='Calculated dose', toolbar_location="above")
    p1.image(image=[calcDose], x=0, y=0, dw=calcDose.shape[1], dh=calcDose.shape[1], palette="Plasma256")

    p2 = figure(plot_width=int(dispSize*2/3), 
                plot_height=int(dispSize*rgbArrL[:,:,0].shape[0]/rgbArrL[:,:,0].shape[1]*2/3), 
                title='Measured dose (Left)', toolbar_location="above")
    p2.image(image=[rgbArrL[:,:,0]], x=0, y=0, dw=rgbArrL[:,:,0].shape[1], dh=rgbArrL[:,:,0].shape[0], palette="Plasma256")

    p3 = figure(plot_width=int(dispSize*2/3), 
                plot_height=int(dispSize*rgbArrC[:,:,0].shape[0]/rgbArrC[:,:,0].shape[1]*2/3), 
                title='Measured dose (Center)', toolbar_location="above")
    p3.image(image=[rgbArrC[:,:,0]], x=0, y=0, dw=rgbArrC[:,:,0].shape[1], dh=rgbArrC[:,:,0].shape[0], palette="Plasma256")

    p4 = figure(plot_width=int(dispSize*2/3), 
                plot_height=int(dispSize*rgbArrR[:,:,0].shape[0]/rgbArrR[:,:,0].shape[1]*2/3), 
                title='Measured dose (Right)', toolbar_location="above")
    p4.image(image=[rgbArrR[:,:,0]], x=0, y=0, dw=rgbArrR[:,:,0].shape[1], dh=rgbArrR[:,:,0].shape[0], palette="Plasma256")

    p5 = figure(plot_width=dispSize, plot_height=int(dispSize*rArrL_crop.shape[0]/rArrL_crop.shape[1]), 
               title='Measured dose (Left)', toolbar_location="above")
    p5.image(image=[rArrL_crop], x=0, y=0, dw=rArrL_crop.shape[1], dh=rArrL_crop.shape[0], palette="Plasma256")

    p6 = figure(plot_width=dispSize, plot_height=int(dispSize*rArrC_crop.shape[0]/rArrC_crop.shape[1]), 
               title='Measured dose (Center)', toolbar_location="above")
    p6.image(image=[rArrC_crop], x=0, y=0, dw=rArrC_crop.shape[1], dh=rArrC_crop.shape[0], palette="Plasma256")

    p7 = figure(plot_width=dispSize, plot_height=int(dispSize*rArrR_crop.shape[0]/rArrR_crop.shape[1]), 
               title='Measured dose (Right)', toolbar_location="above")
    p7.image(image=[rArrR_crop], x=0, y=0, dw=rArrR_crop.shape[1], dh=rArrR_crop.shape[0], palette="Plasma256")
    
    grid1 = gridplot([[p2,p3,p4]])
    grid2 = gridplot([[p1,p5],[p6,p7]])

    show(grid1)
    show(grid2)


   >>> dOSE rEAD !
   >>> gAFCHROMIC fILMS rEAD !


In [10]:
# RIGID REGISTRATION USING SITK:


# Disp images?
visu = False
dispSize = 450

sizeX = rArrL.shape[1]
sizeStep = int(sizeX/3)


# Register dose images and unifies dose maps:
calcDose_L = registerImgs(rArrL_crop, calcDose, nbHistoBins=10, nbiter=10000, convMinValue=1e-9)
calcDose_C = registerImgs(rArrC_crop, calcDose, nbHistoBins=10, nbiter=10000, convMinValue=1e-9)
calcDose_R = registerImgs(rArrR_crop, calcDose, nbHistoBins=10, nbiter=10000, convMinValue=1e-9)

print('   >>> rEGISTRATION dONE!')


# unifies the 3 dose images:
doseArr = np.zeros((m_rectL[1]-m_rectL[0],rArrL.shape[1]))

doseArr[:,0:sizeStep] = calcDose_L[:, 0:sizeStep]
offset = sizeStep - m_rectC[2]
doseArr[:,sizeStep:2*sizeStep] = calcDose_C[:, offset:offset+sizeStep]
doseArr[:,2*sizeStep:] = calcDose_R[:, -sizeStep:]


# unifies the 3 rgb images:
rArr = np.zeros((m_rectL[1]-m_rectL[0],rArrL.shape[1]))
gArr = np.zeros((m_rectL[1]-m_rectL[0],rArrL.shape[1]))
bArr = np.zeros((m_rectL[1]-m_rectL[0],rArrL.shape[1]))

# Le probleme ici est que les matrices R, G et B sont les memes...


rArr[:,0:sizeStep] = rArrL[m_rectL[0]:m_rectL[1],0:sizeStep]
gArr[:,0:sizeStep] = gArrL[m_rectL[0]:m_rectL[1],0:sizeStep]
bArr[:,0:sizeStep] = bArrL[m_rectL[0]:m_rectL[1],0:sizeStep]

rArr[:,sizeStep:2*sizeStep] = rArrC[m_rectC[0]:m_rectC[1],sizeStep:2*sizeStep]
gArr[:,sizeStep:2*sizeStep] = gArrC[m_rectC[0]:m_rectC[1],sizeStep:2*sizeStep]
bArr[:,sizeStep:2*sizeStep] = bArrC[m_rectC[0]:m_rectC[1],sizeStep:2*sizeStep]

rArr[:,2*sizeStep:] = rArrR[m_rectR[0]:m_rectR[1],2*sizeStep:]
gArr[:,2*sizeStep:] = gArrR[m_rectR[0]:m_rectR[1],2*sizeStep:]
bArr[:,2*sizeStep:] = bArrR[m_rectR[0]:m_rectR[1],2*sizeStep:]

print('   >>> rGB iMGS uNIFICATION dONE!')



# shows dose images:
if visu:
    p1 = figure(plot_width=int(dispSize), 
                plot_height=int(dispSize*rArr.shape[0]/rArr.shape[1]), 
                title='R values', toolbar_location="above")
    p1.image(image=[rArr], x=0, y=0, dw=rArr.shape[1], dh=rArr.shape[0], palette="Plasma256")

    p2 = figure(plot_width=int(dispSize), 
                plot_height=int(dispSize*gArr.shape[0]/gArr.shape[1]), 
                title='G values', toolbar_location="above")
    p2.image(image=[gArr], x=0, y=0, dw=gArr.shape[1], dh=gArr.shape[0], palette="Plasma256")

    p3 = figure(plot_width=int(dispSize), 
                plot_height=int(dispSize*bArr.shape[0]/bArr.shape[1]), 
                title='B values', toolbar_location="above")
    p3.image(image=[bArr], x=0, y=0, dw=bArr.shape[1], dh=bArr.shape[0], palette="Plasma256")

    p4 = figure(plot_width=int(dispSize), 
                plot_height=int(dispSize*doseArr.shape[0]/doseArr.shape[1]), 
                title='Calculated dose', toolbar_location="above")
    p4.image(image=[doseArr], x=0, y=0, dw=doseArr.shape[1], dh=doseArr.shape[0], palette="Plasma256")

    grid1 = gridplot([[p1,p2],[p3,p4]])

    show(grid1)



   >>> rEGISTRATION dONE!
   >>> rGB iMGS uNIFICATION dONE!


In [ ]:
# COMPARE REGISTERED IMAGES:

arr1 = np.nan_to_num(rArr)
# min1 = np.amin(arr1)
# max1 = np.amax(arr1)
# arr1 = (arr1-min1)/(max1-min1)

arr2 = np.nan_to_num(doseArr)
# min2 = np.amin(arr2)
# max2 = np.amax(arr2)
# arr2 = (arr2-min2)/(max2-min2)

max1 = np.amax(arr1)
max2 = np.amax(arr2)
arr1 = arr1/max1*max2

compare2Imgs(arr1, arr2, 150, 80, 
             title1="film", title2='dose')

# RBGB Calibration curves and conversion to dose:

In [44]:
# CALIBRATION MODEL : 


# fitting functions for calibration:
def funcDose(x, a, b, c, d, e):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**c*x[:,1]**d) + e


# params:
beginLine = 30
endLine = 205

beginColumn = 20
# columnOffset = 1
# nbOfColumns = 270
# columnsHalfWidth = 0
columnOffset = 270
nbOfColumns = 1
columnsHalfWidth = 2


# R/B and G/B calculation:
array_rb = rArr / bArr
array_gb = gArr / bArr


# Model creation:
gafModelsDict = {"RBGBDoseCurve":[]}

for i in range(nbOfColumns):
    paramsDose = createLineCal_RBGBModel(array_rb, array_gb, doseArr, beginColumn + i*columnOffset,
                                                  beginLine, endLine, w=columnsHalfWidth, 
                                                  dispParameters=False, dispGraphs=False)
    gafModelsDict["RBGBDoseCurve"].append(paramsDose.tolist())

    
print("   >>> mODEL dONE !")

   >>> mODEL dONE !


In [45]:
# CONVERSION TO DOSE:
# <!> this does not work with a new file, it must be cropped at the same position and size as the 
# calibration file.
# This is for test only, and needs to be changed to be used in every situation
# One of the main problem is that the registration does not work if the measured film is not cropped
#
# It might be interesting to change value outside the film or remove any dose below 300 cGy...



# R/B and G/B calculation:
array_rb = rArr / bArr
array_gb = gArr / bArr


doseImg = np.zeros(array_rb.shape)

width = array_rb.shape[1]
for i in range(width):
    
    # model selection:
    index = int((i-beginColumn)/columnOffset-0.5)
    if index<0: index=0
    elif index>=nbOfColumns : index=nbOfColumns-1
    
    mod = gafModelsDict["RBGBDoseCurve"][index]
    
    rbgb = np.zeros((array_rb.shape[0],2))
    rbgb[:,0] = array_rb[:,i]
    rbgb[:,1] = array_gb[:,i]

    doseImg[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])

    

compare2Imgs(doseImg, doseArr, 100, 150, title1="Gafchromic dose", title2="TPS dose", visibleLegend = True)






In [42]:
# CONVERSION TO DOSE:
# <!> this does not work with a new file, it must be cropped at the same position and size as the 
# calibration file.
# This is for test only, and needs to be changed to be used in every situation
# One of the main problem is that the registration does not work if the measured film is not cropped
#
# It might be interesting to change value outside the film or remove any dose below 300 cGy...




# Params:

# path = 'G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/testLateral/'
# nbOfFiles = 1
# firstNb = 3

# filesName = "test-pos01_0"
# gaf_rect = [100,170,0,120]

# filesName = "test-pos03_0"
# gaf_rect = [100,170,50,170]

path = 'G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/testAmbulance14avril2021/'
nbOfFiles = 1
firstNb = 5
# filesName = "scan 29h30/pos01_0"
# gaf_rect = [230,350,30,170]
filesName = "scan 29h30/pos02_0"
gaf_rect = [230,350,110,250]
# filesName = "scan 29h30/pos03_0"
# gaf_rect = [230,350,170,310]

fileExtension = ".tif"

doseFileName = "RTplan&dose/RDambulance.dcm"
planFileName =  "RTplan&dose/RPambulance.dcm"


# Reads the DICOMRT dose file:
(tPSdose, dimx, dimy, pixsizex, pixsizey) = readDoseMatrix_AxialPlane_Isocenter(
    path+doseFileName, path+planFileName, offset=0, verbose=False)


# Reads the Gafchromic file:
try:
    g = Radiochromic_RB(path+filesName, firstNb, nbOfFiles)
    g.subSampleDataArray(10)
    gAFdose_RB = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    rgbArr = g.getArray()
except ValueError as err:
    print('Erreur: ' + err)


# R/B and G/B calculation:
array_rb = rgbArr[:,:,0] / rgbArr[:,:,2]
array_gb = rgbArr[:,:,1] / rgbArr[:,:,2]

gAFdose_RBGB = np.zeros(array_rb.shape)
width = array_rb.shape[1]
for i in range(width):
    
    # model selection:
    index = int((i-beginColumn)/columnOffset-0.5)
    if index<0: index=0
    elif index>=nbOfColumns : index=nbOfColumns-1
    
    mod = gafModelsDict["RBGBDoseCurve"][index]
    
    rbgb = np.zeros((array_rb.shape[0],2))
    rbgb[:,0] = array_rb[:,i]
    rbgb[:,1] = array_gb[:,i]

    gAFdose_RBGB[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])
 

print("   >>> dONE")

   >>> dONE


In [43]:
# Visualisation des résultats:
threshold = 0
scaleFactor = 1.0

visuGafDoseRBGB = gAFdose_RBGB[gaf_rect[0]:gaf_rect[1], gaf_rect[2]:gaf_rect[3]]
visuGafDoseRBGB[visuGafDoseRBGB<threshold] = 0

visuGafDoseRB = gAFdose_RB[gaf_rect[0]:gaf_rect[1], gaf_rect[2]:gaf_rect[3]]
visuGafDoseRB[visuGafDoseRB<threshold] = 0


visuCalcDose = tPSdose#np.rot90(tPSdose)
visuCalcDose[visuCalcDose<threshold] = 0
visuCalcDose = registerImgs(visuGafDoseRBGB, visuCalcDose, nbHistoBins=20, nbiter=100000, convMinValue=1e-10)


compare2Imgs(visuGafDoseRB, visuCalcDose*scaleFactor, 75, 60, title1="R/B", title2="TPS dose", visibleLegend = True)
compare2Imgs(visuGafDoseRBGB, visuCalcDose*scaleFactor, 75, 60, title1="COCAPI", title2="TPS dose", visibleLegend = True)

In [17]:
# Variables:
path = 'G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/testAmbulance14avril2021/'
nbOfFiles = 1
firstNb = 5
fileExtension = ".tif"

imgNames = ["scan 29h30/pos01_0", "scan 29h30/pos02_0", "scan 29h30/pos03_0"]

rect_imgs = [[230,350,30,170],
            [230,350,110,250],
            [230,350,170,310]]

dosemax = 800

subfactor = 10
dimViewer = 480
regImgNbToDisp = 6
line = 50
col = 50

doseFileName = "RTplan&dose/RDambulance.dcm"
planFileName =  "RTplan&dose/RPambulance.dcm"


# Reads the DICOMRT dose file:
(tPSdose, dimx, dimy, pixsizex, pixsizey) = readDoseMatrix_AxialPlane_Isocenter(
    path+doseFileName, path+planFileName, offset=0, verbose=False)


# Reads the first gafchromic file:
try:
    g = Radiochromic_RB(path+imgNames[0], firstNb, nbOfFiles)
    g.subSampleDataArray(10)
    gAFdose_RB = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    rgbArr = g.getArray()
except ValueError as err:
    print('Erreur: ' + err)

# R/B and G/B calculation:
array_rb = rgbArr[:,:,0] / rgbArr[:,:,2]
array_gb = rgbArr[:,:,1] / rgbArr[:,:,2]

gAFdose_RBGB = np.zeros((array_rb.shape[0], array_rb.shape[1], len(imgNames)))
width = array_rb.shape[1]
for i in range(width):
    # model selection:
    index = int((i-beginColumn)/columnOffset-0.5)
    if index<0: index=0
    elif index>=nbOfColumns : index=nbOfColumns-1
    
    mod = gafModelsDict["RBGBDoseCurve"][index]
    
    rbgb = np.zeros((array_rb.shape[0],2))
    rbgb[:,0] = array_rb[:,i]
    rbgb[:,1] = array_gb[:,i]
    
    gAFdose_RBGB[:,i,0] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])


# Reads all other gafchromic files:
for j in range(1,len(imgNames)):
    
    try:
        g = Radiochromic_RB(path+imgNames[j], firstNb, nbOfFiles)
        g.subSampleDataArray(10)
        gAFdose_RB = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
        rgbArr = g.getArray()
    except ValueError as err:
        print('Erreur: ' + err)

    # R/B and G/B calculation:
    array_rb = rgbArr[:,:,0] / rgbArr[:,:,2]
    array_gb = rgbArr[:,:,1] / rgbArr[:,:,2]

    for i in range(width):
        # model selection:
        index = int((i-beginColumn)/columnOffset-0.5)
        if index<0: index=0
        elif index>=nbOfColumns : index=nbOfColumns-1
    
        mod = gafModelsDict["RBGBDoseCurve"][index]
    
        rbgb = np.zeros((array_rb.shape[0],2))
        rbgb[:,0] = array_rb[:,i]
        rbgb[:,1] = array_gb[:,i]

        gAFdose_RBGB[:,i,j] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])

print('   >>> dOSE iMAGES cALCULATED !')


   >>> dOSE iMAGES cALCULATED !


In [18]:
# Variables:
colors = ['firebrick', 'darkblue', 'green', 'purple', 'yellow']
visu = True
regImgNbToDisp = 0
dimViewer = 450

line = 60
col = 70


# images:
visuGafDoseRBGB = np.zeros((gaf_rect[1]-gaf_rect[0], gaf_rect[3]-gaf_rect[2], len(imgNames)))
visuGafDoseRBGB[:,:,0] = gAFdose_RBGB[rect_imgs[0][0]:rect_imgs[0][1], rect_imgs[0][2]:rect_imgs[0][3], 0]

for i in range(1,len(imgNames)):
    tmpImg = gAFdose_RBGB[rect_imgs[i][0]:rect_imgs[i][1], rect_imgs[i][2]:rect_imgs[i][3], i]
    visuGafDoseRBGB[:,:,i] = registerImgs(visuGafDoseRBGB[:,:,0], tmpImg, nbHistoBins=20, 
                                          nbiter=100000, convMinValue=1e-9)
    
visuGafDoseRBGB[visuGafDoseRBGB<threshold] = 0

visuCalcDose = tPSdose#np.rot90(tPSdose)
visuCalcDose[visuCalcDose<threshold] = 0
visuCalcDose = registerImgs(visuGafDoseRBGB[:,:,0], visuCalcDose, nbHistoBins=20, nbiter=100000, convMinValue=1e-9)


# Visualization:    
if visu:
    img1 = visuCalcDose
    img2 = visuGafDoseRBGB[:,:,regImgNbToDisp]
    imgsProfile = visuGafDoseRBGB
    
    # Img 1:
    p1 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img1.shape[0]/img1.shape[1]), 
                title='Reference image', toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img2.shape[0]/img2.shape[1]), 
               title='Registered image', toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=3, line_color='black', legend_label='Reference Image')
    for i in range(len(imgNames)):
        p3.line(x3, imgsProfile[line,:,i], line_width=1, line_color=colors[i], legend_label='pos no'+ str(i))
    
    p3.legend.click_policy="hide"
    p3.legend.location = "bottom_left"


    # Vertical profile:
    p4 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color='black', legend_label='Reference Image')
    for i in range(len(imgNames)):
        p4.line(x4, imgsProfile[:,col,i], line_width=2, line_color=colors[i], legend_label='pos no'+ str(i))

    p4.legend.click_policy="hide"
    p4.legend.location = "bottom_left"

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)
    
print("  >>> mULTIPLE iMAGE tEST dONE !")

  >>> mULTIPLE iMAGE tEST dONE !


# tESTS  multilinéaire

In [67]:
# CALIBRATION MODEL : 


# fitting functions for calibration:
def funcDose(x, a, b, c, d, e):
    return a*x[:,0] + b*x[:,1] + c*x[:,2] + d



# modèle d'une ligne
beginLine = 30
endLine = 205
column = 100
w = 5

# crops the matrices:
if w==0:
    r = rArr[beginLine:endLine, column]
    g = gArr[beginLine:endLine, column]
    b = bArr[beginLine:endLine, column]
    dose = doseArr[beginLine:endLine, column]
else:
    r = np.mean(rArr[beginLine:endLine, column-w:column+w], axis=1)
    g = np.mean(gArr[beginLine:endLine, column-w:column+w], axis=1)
    b = np.mean(bArr[beginLine:endLine, column-w:column+w], axis=1)
    dose = np.mean(doseArr[beginLine:endLine, column-w:column+w], axis=1)

    
# Dose vs RB&GB fitting:
nbLines = r.shape[0]
rgb = np.zeros((nbLines,3))

# rgb[:,0] = r
# rgb[:,1] = g
# rgb[:,1] = b
rgb[:,0] = -np.log10(r/65535.0)
rgb[:,1] = -np.log10(g/65535.0)
rgb[:,1] = -np.log10(b/65535.0)

paramsDose, params_covDose = optimize.curve_fit(funcDose, rgb, dose)


# Display parameters models:
print('Parameters Dose vs RB & GB:')
for i in range(len(paramsDose)):
    print('   - parameter', i, ':', paramsDose[i])

# Display graph results
p2 = figure(plot_width=300, plot_height=int(300*rArr.shape[0]/rArr.shape[1]), 
            title='', toolbar_location="above")

p2.image(image=[rArr], x=0, y=0, dw=rArr.shape[1], dh=rArr.shape[0], palette="Plasma256")
p2.line((column, column), (beginLine, endLine), line_alpha=0.7, line_color="white")

p3 = figure(plot_width=600, plot_height=600, title="R, G and B vs dose", toolbar_location="above")
p3.circle(r, dose, size=3, color='firebrick', alpha=0.3, legend_label="R") 
p3.circle(g, dose, size=3, color='darkgreen', alpha=0.3, legend_label="G") 
p3.circle(b, dose, size=3, color='blue', alpha=0.3, legend_label="B") 

p4 = figure(plot_width=450, plot_height=450, title="Dose calculated vs real", toolbar_location="above")
p4.line(dose, funcDose(rgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], paramsDose[4]), 
        line_width=2, line_color='firebrick')
p4.line(dose, dose, line_width=1, line_color='black', line_dash='dotted')

p5 = figure(plot_width=450, plot_height=450, title="Dose calculation Error in %", toolbar_location="above")
p5.line(dose,(dose-funcDose(rgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], \
                                    paramsDose[4]))/dose*100, line_width=2, line_color='firebrick')
p5.line(dose,2 , line_width=1, line_color='black', line_dash='dotted')
p5.line(dose,-2 , line_width=1, line_color='black', line_dash='dotted')

grid1 = gridplot([[p3, p2]])
grid2 = gridplot([[p4,p5]])

show(grid1)
show(grid2)


    
print("   >>> mODEL dONE !")

Parameters Dose vs RB & GB:
   - parameter 0 : -2108.8278085305146
   - parameter 1 : 8447.049255173282
   - parameter 2 : 1.0
   - parameter 3 : -2080.2171562772055
   - parameter 4 : 1.0


   >>> mODEL dONE !
